In [1]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 10.9 MB/s 
     |████████████████████████████████| 67 kB 744 kB/s 
     |████████████████████████████████| 6.8 MB 19.4 MB/s 
     |████████████████████████████████| 895 kB 37.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import numpy as np

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import torch
from torch.utils.data import (TensorDataset, DataLoader,
                              RandomSampler, SequentialSampler)
from transformers import BertTokenizer, BertConfig
from transformers import BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from distutils.version import LooseVersion as LV
from sklearn.model_selection import train_test_split
import io

#path = r'D:\tirocinioLC\tirocinioLC'
from google.colab import drive
drive.mount('/content/drive')
directory = '/content/drive/My Drive/Text Analytics/Data/'

Mounted at /content/drive


In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    devicename = '['+torch.cuda.get_device_name(0)+']'
else:
    device = torch.device('cpu')
    devicename = ""
    
print('Using PyTorch version:', torch.__version__,
      'Device:', device, devicename)
assert(LV(torch.__version__) >= LV("1.0.0"))

Using PyTorch version: 1.10.0+cu111 Device: cuda [Tesla K80]


Import dataset

In [4]:
df = pd.read_json(directory+'cleandf3.json')

df.Data = pd.to_datetime(df.Data)
df = df[df.Data.dt.year < 2019]
df = df.sample(frac=0.85, random_state=42)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32039 entries, 2840 to 29371
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID                  32039 non-null  int64         
 1   Titolo_Rif          32037 non-null  object        
 2   Rating              32039 non-null  object        
 3   Nome_Autore         32039 non-null  object        
 4   Data                32039 non-null  datetime64[ns]
 5   Numero_Capitoli     32039 non-null  int64         
 6   Racconto_Text_Only  32039 non-null  object        
 7   N_Tot_Rec           32039 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 2.2+ MB
None


,ID,Titolo_Rif,Rating,Nome_Autore,Data,Numero_Capitoli,Racconto_Text_Only,N_Tot_Rec
2840,59819,Tutto Per Una Meravigliosa Partita di Quidditch,verde,Gillywater,2005-11-19,1,Buon Compleanno Love! ...,12
33471,1135350,Dopotutto Babbanologia non è inutile...,verde,Gils_Malfoy,2012-06-29,1,AVVERTENZE! Prima di iniziare a leggere è bene...,7
34947,1260606,Non piangere.,verde,A n o n y m o u s Rei,2012-11-09,1,"Ti tappi un altro po’ le orecchie, Theo. Fa...",2
35908,1407567,Una finestra sul passato,verde,Ma_AiLing,2012-11-24,1,Il professor Lupin si stava dirigendo nel suo ...,2
22312,690486,Qualcuno uccida Potter,verde,lotti_,2011-06-04,1,"Un raggio di sole scarlatto, tenue e delicata ...",4


In [5]:
df = df[df['Rating']!='giallo']
df = df[df['Rating']!='arancione']
########SAMPLE (MEMORY ISSUES)
df_rosso = df[df['Rating']=='rosso']
df_verde = df[df['Rating']=='verde']
print(len(df_verde))
df_verde = df_verde.sample(frac=0.85, random_state=42)
print(len(df_verde))

df = pd.concat([df_rosso, df_verde])
df.head()

22223
18890


,ID,Titolo_Rif,Rating,Nome_Autore,Data,Numero_Capitoli,Racconto_Text_Only,N_Tot_Rec
36795,1538089,Professor Potter,rosso,LadyDaredevil,2013-01-15,1,A LadyDepp :) PROFESSOR POTTER La gr...,10
8972,178294,tra ron e grattastinchi,rosso,JoeMarch,2007-10-11,1,"TRA RON E GRATTASTINCHI ""no"" ""eddai hermione...",3
51219,3534405,Love always remains,rosso,Alex91,2016-09-17,1,@page p a: link Una ...,3
8740,172344,L'ultima alba,rosso,alicesimone,2007-10-16,1,La Seconda Guerra aveva ormai raggiunto il suo...,8
50828,3494175,A thousand kisses - Lo Specchio delle Anime,rosso,Marne,2016-07-20,1,Comunicazione di servizio:...,5


In [6]:
target = df[['Rating']]
del df['Rating']

In [7]:
x_test = pd.read_csv(directory+'x_test_final1v1.csv', index_col=0)
y_test = pd.read_csv(directory+'y_test_final1v1.csv', index_col=0)

In [8]:
x_test

,ID,Titolo_Rif,Nome_Autore,Data,Numero_Capitoli,Racconto_Text_Only,N_Tot_Rec
29257,901391,"Ti amo Hermione, come non amerò mai nessun'altra.",misanthrop,2011-12-23,1,"Hey Hermione, qui è Ron che scrive. Sai perch...",1
23940,751036,Je t'adore,esistere,2011-02-07,1,Je t'adore Je t'adore Fandom: Ha...,4
45937,2870666,Di incontri e Di Zucchero,DarkLullaby21,2014-10-15,1,Di Incontri e di Zucchero La stanza delle f...,1
53440,3743913,Remorsus,Panna_Malfoy,2018-10-02,1,"Tic, tic, tic. Ormai ne aveva contati esatta...",3
28337,872244,Eppure mi hai cambiato la vita,M4RT1,2011-11-20,1,Mi hai cambiato la vita. E pensare che ero io...,4
...,...,...,...,...,...,...,...
34936,1262226,Primo appuntamento,Erp6294,2012-12-09,1,Era una giornata soleggiata. Hermione era molt...,5
31025,979384,L'ennesima Umiliazione,_Kidagakash_,2012-05-03,1,drabble_110 ...,7
28185,864340,Certi amori ti rendono cenere.,Lue,2011-11-11,1,Certi amori ti rendono cenere. Si rinco...,3
19757,610016,Magie e miracoli,Hi Ban,2010-02-12,1,Magie e miracoli “Che cos’hai intenzione ...,2


Text Preparation

In [9]:
sentences_test = ["[CLS] " + str(s) for s in x_test.Racconto_Text_Only]
labels_test  = [1 if (value=='rosso') else 0 for value in y_test.Rating]

In [10]:
labels_test

[0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,


In [11]:
from transformers import AutoModel, AutoTokenizer

model_name = "dbmdz/bert-base-italian-xxl-cased"

tokenizer = BertTokenizer.from_pretrained(model_name)

model = AutoModel.from_pretrained(model_name)

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-italian-xxl-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
from tqdm import tqdm

In [13]:
#tokenized_test  = [tokenizer.tokenize(s) for s in tqdm(sentences_test)]

In [14]:
import pickle
#with open(directory+r'RATING1v1tokenized_test_FINAL.pkl','wb') as outfile:
#  pickle.dump(tokenized_test, outfile)

In [15]:
b_file = open(directory+r"RATING1v1tokenized_test_FINAL.pkl", "rb")
tokenized_test = pickle.load(b_file)

In [16]:
len(tokenized_test)

6189

In [17]:
MAX_LEN_test = 125
K = MAX_LEN_test

tokenized_test = [t[((len(t) // 2) - (K // 2)) : ((len(t) // 2) + (K // 2) + 1) ]+['SEP'] for t in tokenized_test]
tokenized_test = [["[CLS]"] + s for s in tokenized_test]

print ("The truncated tokenized first test sentence:")
print (tokenized_test[0])

The truncated tokenized first test sentence:
['[CLS]', 'di', 'affon', '##dare', 'una', 'mano', 'nei', 'tuoi', 'morbi', '##di', 'capelli', 'e', 'l', "'", 'altra', 'posa', '##ta', 'sulla', 'tua', 'schiena', 'strin', '##gendo', '##ti', '.', 'So', '##gno', 'il', 'tuo', 'profumo', 'alla', 'fra', '##gola', ',', 'sogno', 'il', 'tuo', 'petto', 'contro', 'il', 'mio', 'e', 'il', 'tuo', 'cuore', 'che', 'batte', 'accanto', 'al', 'mio', '.', 'So', '##gno', 'di', 'asciuga', '##re', 'le', 'tue', 'lacrime', ',', 'per', 'non', 'veder', '##le', 'mai', 'più', 'ba', '##gnare', 'il', 'tuo', 'viso', '.', 'So', '##gno', 'di', 'appoggiare', 'lieve', '##mente', 'le', 'tue', 'labbra', 'contro', 'le', 'mie', 'e', 'bacia', '##rti', 'nel', 'modo', 'più', 'dolce', 'possibile', '.', 'Poi', 'sorri', '##derti', 'e', 'vedere', 'il', 'tuo', 'sorriso', 'che', 'mi', 'fa', 'sciogliere', ',', 'che', 'mi', 'rie', '##mpe', 'il', 'cuore', 'di', 'felicità', '.', 'Poi', 'dirtelo', '.', "'", "'", 'Her', '##mio', '##ne', ',', 'io'

In [ ]:
ids_test = [tokenizer.convert_tokens_to_ids(t) for t in tokenized_test]
ids_test = np.array([np.pad(i, (0, 128-len(i)),
                            mode='constant') for i in ids_test])

print ("The indices of the first test sentence:")
print (ids_test[0])

The indices of the first test sentence:
[  120 19063  1992   224  1865   713  2300  9097   195  4896   126   181
  1553  5241 13806   115   602   930 11131 10073  5736   116   697   756
   371   162   637 13501   334   731  8057  1307  6107   162   637 11399
   593   162   619   126   162   637  2214   158  6585  6481   157   619
   697   756   371   120  9583   113   199  3557 16903  1307   156   212
  3273   125   779   353   378  3902   162   637 10248   697   756   371
   120 21416 13862   255   199  3557 15816   593   199  1388   126 15143
  2108   207   739   353  5311  1244   697  3844  8517  5198   126  1513
   162   637 15216   158   318   231 30178  1307   158   318  5040  5889
   162  2214   120 13153   697  3844 14288   697  1553  1553  5905  2601
   363  1307   642   364  4714   101     0     0]


In [ ]:
amasks_test = []

for seq in ids_test:
  seq_mask = [float(i>0) for i in seq]
  amasks_test.append(seq_mask)

In [ ]:
labels_test  = [1 if (value=='rosso') else 0 for value in y_test.Rating]
test_inputs = torch.tensor(ids_test)
test_labels = torch.tensor(labels_test)
test_masks  = torch.tensor(amasks_test)

In [ ]:
BATCH_SIZE = 32

print('Test: ', end="")
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler,
                             batch_size=BATCH_SIZE)
print(len(test_data), 'documents')

Test: 6189 documents


# MODIFICHE

In [ ]:
import torch.nn as nn
from sklearn.metrics import f1_score

In [ ]:
from tabulate import tabulate

def evaluate(loader, loss_vector=None, test=False):
    model.eval()

    n_correct, n_all = 0, 0
    TP = 0
    TN = 0
    FP = 0
    FN = 0

    b_labels_v2 = torch.tensor([],dtype=torch.long)
    b_input_ids_v2 = torch.tensor([], dtype=torch.long)
    b_attn_mask_v2 = torch.tensor([], dtype=torch.long)

    batchcount = 0
    for batch in loader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        b_labels_v2 = torch.cat((b_labels_v2.to(device), b_labels), 0)
        b_input_ids_v2 = torch.cat((b_input_ids_v2.to(device), b_input_ids), 0)
        b_attn_mask_v2 = torch.cat((b_attn_mask_v2.to(device), b_input_mask), 0)

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None,
                          attention_mask=b_input_mask)
            logits = outputs.logits

        logits = logits.detach().cpu()#.numpy()

        #Predictions
        predictions = np.argmax(logits.numpy(), axis=1)
        #Labels
        labels = b_labels.to('cpu').numpy()

        n_correct += np.sum(predictions == labels)
        n_all += len(labels)

        for truth, pred in zip(labels, predictions):
          if truth == 0 and pred == 0:
            TN +=1
          elif truth == 1 and pred == 1:
            TP +=1
          elif truth == 1 and pred == 0:
            FN +=1
          elif truth ==0 and pred == 1:
            FP +=1

        if test:
          batchcount += 1
          #print("Ho preso", batchcount, 'batches su ?')


    print('Accuracy: [{}/{}] {:.4f}'.format(n_correct, n_all,
                                          n_correct/n_all))

    if (TP+FP!=0):
      precision_1 = (TP) / (TP+FP)
    else:
      precision_1 = 0

    if (TP+FN!=0):
      recall_1 = (TP) / (TP+FN)
    else:
      recall_1 = 0

    precision_0 = TN / (TN+FN)
    recall_0 = TN / (TN+FP)

    f1_0 = 2* ((precision_0*recall_0) / (precision_0+recall_0))
    if (precision_1+recall_1) != 0:
      f1_1 = 2* ((precision_1*recall_1) / (precision_1+recall_1))
    else:
      f1_1 = 0

    data = [['verde', precision_0, recall_0, f1_0],
            ['rosso', precision_1, recall_1, f1_1]]

    data2 = [['predROSSO(1)', TP, FP], 
             ['predVERDE(0)', FN, TN]]

    print(tabulate(data, headers=["Class", "Precision", "Recall", "F1"]))
    print()
    print(tabulate(data2, headers=['', 'truthROSSO', 'truthVERDE']))



Evaluation

LOAD MODEL

In [ ]:
c_file = open(directory+r"XXXMODEL_RATING_1v1_2_1_3epoche.pkl", "rb")
model = pickle.load(c_file)

In [ ]:
torch.cuda.empty_cache()
print('Test set:')
evaluate(test_dataloader, test=True)

Test set:
Accuracy: [5820/6189] 0.9404
Class      Precision    Recall        F1
-------  -----------  --------  --------
verde       0.952975  0.983413  0.967955
rosso       0.72434   0.47318   0.572422

                truthROSSO    truthVERDE
------------  ------------  ------------
predROSSO(1)           247            94
predVERDE(0)           275          5573
